In [1]:
# this is a copy of tutorial 3 at the start
# but instead of faking it by dragging the car through waypoints 
# we will be giving the car simple inputs to guide it along the route

In [2]:
# all tsports 
import carla # the sim library itself 
import time # to set a delay after each photo 
import cv2 # to work with images from cameras
import numpy as np # in this example to change image representation re-shaping 
import math

In [3]:
#connect to the sim

client = carla.Client('localhost', 2000)

In [7]:
# Define the environment/world and get possible spawn points
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)  # Set timeout to connect to the server
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

# Look for a blueprint of a Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

# Select a spawn point and spawn the vehicle
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

# Check if the vehicle was spawned successfully
if vehicle is None:
    print("Failed to spawn vehicle. Check spawn point or blueprint.")
else:
    print("Vehicle spawned successfully!")

# Camera mount offset on the car
CAMERA_POS_Z = 3.0
CAMERA_POS_X = -5.0

# Setting up the RGB camera
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))

# Create the camera in the simulator and attach it to the vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# Camera callback function to process images
def camera_callback(image, data_dict):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    reshaped = array.reshape((image.height, image.width, 4))  # BGRA format
    data_dict['image'] = reshaped

# Camera resolution attributes
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

# Initialize the camera data dictionary
camera_data = {'image': np.zeros((image_h, image_w, 4))}

# Start the camera and listen for images
camera.listen(lambda image: camera_callback(image, camera_data))
print("Camera is streaming live images.")


Failed to spawn vehicle. Check spawn point or blueprint.
Camera is streaming live images.


In [8]:
# Define speed constants
PREFERRED_SPEED = 20  # Target speed in some units (e.g., km/h or m/s)
SPEED_THRESHOLD = 2  # Speed tolerance before adjustments

# Adding parameters to display text on the image
font = cv2.FONT_HERSHEY_SIMPLEX  # Font type for OpenCV
# Defining positions for telemetry display on the screen
org_speed = (30, 30)  # Current speed display position
org_steering = (30, 50)  # Future steering angle display position
org_telemetry = (30, 70)  # Other telemetry outputs

# Text properties
fontScale = 0.8
color = (255, 255, 255)  # White color for text
thickness = 2  # Line thickness for text

# Function to maintain speed
def maintain_speed(current_speed):
    """
    Adjust throttle to maintain desired speed.

    Args:
        current_speed (float): The current speed of the vehicle.

    Returns:
        float: Throttle value to adjust speed.
    """
    if current_speed < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8  # Full gas (adjust this value as needed)
    elif current_speed > PREFERRED_SPEED + SPEED_THRESHOLD:
        return 0.4  # Gentle throttle to slow down
    else:
        return 0.6  # Maintain steady speed

# Example usage of displaying telemetry on an image
def display_telemetry(image, current_speed, steering_angle):
    """
    Displays telemetry data on an image.

    Args:
        image (numpy array): The image where telemetry is displayed.
        current_speed (float): The current speed of the vehicle.
        steering_angle (float): The current steering angle of the vehicle.

    Returns:
        numpy array: The image with telemetry overlaid.
    """
    speed_text = f"Speed: {current_speed:.2f}"
    steering_text = f"Steering: {steering_angle:.2f}"
    telemetry_text = "Telemetry: OK"  # Replace with other telemetry info if needed

    # Put text on the image
    cv2.putText(image, speed_text, org_speed, font, fontScale, color, thickness)
    cv2.putText(image, steering_text, org_steering, font, fontScale, color, thickness)
    cv2.putText(image, telemetry_text, org_telemetry, font, fontScale, color, thickness)

    return image


In [7]:
# CARLA Driving Straight Demo with Camera Sensor

import carla
import numpy as np
import cv2
import math

# Step 1: Connect to CARLA server and get world object
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# Step 2: Spawn a vehicle in the simulation world
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('model3')[0]
spawn_points = world.get_map().get_spawn_points()
spawn_point = spawn_points[0] if spawn_points else carla.Transform()
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

# Step 3: Initialize camera data storage
camera_data = {"image": None}

# Step 4: Define a callback function to process images from the camera sensor
def process_image(image):
    """
    Callback function to process images from the CARLA camera sensor.
    Converts raw image data into a NumPy array compatible with OpenCV.
    """
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))  # BGRA format
    camera_data["image"] = array[:, :, :3]  # Keep only BGR channels (discard alpha channel)

# Step 5: Attach a camera sensor to the vehicle
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '800')  # Width of the image
camera_bp.set_attribute('image_size_y', '600')  # Height of the image
camera_bp.set_attribute('fov', '90')           # Field of view in degrees

spawn_point_camera = carla.Transform(carla.Location(x=1.5, z=2.4))  # Camera position relative to vehicle
camera = world.spawn_actor(camera_bp, spawn_point_camera, attach_to=vehicle)
camera.listen(lambda image: process_image(image))

# Step 6: Main loop for driving straight demo
cv2.namedWindow("RGB Camera", cv2.WINDOW_AUTOSIZE)
quit_demo = False

while not quit_demo:
    world.tick()  # Advance the simulation by one tick

    # Apply throttle control to make the vehicle move forward
    vehicle.apply_control(carla.VehicleControl(throttle=0.5))  # Throttle set to 50%

    # Check for user input to quit ('q' key)
    if cv2.waitKey(1) == ord('q'):
        quit_demo = True

    # If image data is available from the camera, process and display it
    if camera_data["image"] is not None:
        image = camera_data["image"].copy()  # Make a writable copy of the image

        # Get current velocity of the vehicle (returns a vector)
        v = vehicle.get_velocity()

        # Calculate speed in km/h using vector magnitude and conversion factor (3.6)
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)

        # Overlay speed information on the image using OpenCV text functions
        font = cv2.FONT_HERSHEY_SIMPLEX
        org_speed = (50, 50)      # Position of text on screen (x, y)
        fontScale = 1             # Size of text
        color = (0, 255, 0)       # Text color (green)
        thickness = 2             # Thickness of text

        # Add speed text overlay to the image
        image = cv2.putText(image, f'Speed: {int(speed)} kmph', org_speed, font,
                            fontScale, color, thickness, cv2.LINE_AA)

        # Display the processed image in an OpenCV window
        cv2.imshow("RGB Camera", image)

# Step 7: Clean up resources after exiting the loop
cv2.destroyAllWindows()  # Close all OpenCV windows
camera.stop()            # Stop listening to camera data

# Destroy all actors in CARLA (vehicles and sensors)
for actor in world.get_actors().filter('vehicle.*'):  # Destroy all vehicles
    actor.destroy()

for sensor in world.get_actors().filter('sensor.*'):  # Destroy all sensors
    sensor.destroy()


In [9]:
# Destroy all actors (vehicles and sensors)
for actor in world.get_actors().filter('vehicle.*'):
    actor.destroy()

for sensor in world.get_actors().filter('sensor.*'):
    sensor.destroy()
